<a href="https://www.kaggle.com/code/waheedhussain2/book-insights-ai-intelligent-q-a-for-pdf-document?scriptVersionId=205069265" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <span style="color:green">Introduction</span>
**Book Insights AI** is a powerful, AI-driven question-answering application designed to help users extract knowledge and insights from PDF books and documents efficiently. By combining the power of Google Generative AI’s Gemini model and Sentence Transformers for semantic search, this application can answer detailed questions about the content of any uploaded PDF. Whether you’re studying, researching, or simply curious, Book Insights AI allows you to interact directly with your documents and get accurate, context-aware responses to your specific questions.

## <span style="color:green">Uses and Benefits </span>
### 1. Comprehensive Information Retrieval
This tool provides users with the capability to retrieve both general summaries and fine-grained information from any section of a PDF book or document, making it ideal for students, researchers, and professionals. Book Insights AI is especially useful for academic and technical documents where detailed answers and specific information are essential.

### 2. Efficient Study Tool
For students and educators, this application can save hours spent reading entire chapters or sections by quickly pulling answers to specific questions. You can use it to extract key points, understand difficult topics, or get summaries to aid in quicker learning.

### 3. Enhanced Research Capabilities
Researchers can use this tool to identify key information in scientific papers, legal documents, or technical manuals. Instead of reading through lengthy texts, users can simply ask questions related to their research, and the application retrieves relevant information from the document’s content.

### 4. Flexible Contextualization
By leveraging semantic search, Book Insights AI understands the intent behind each question, allowing it to give answers with more precision. Even if a document is lengthy, the AI model retrieves contextually appropriate information based on the user's query.

### 5. User-Friendly and Intuitive
The app is designed to be easy to use and accessible, even for those unfamiliar with machine learning or natural language processing. Users simply upload a PDF, ask questions, and receive prompt, accurate responses.

##  <span style="color:green">How to Use</span>
### 1. Upload a PDF Document
Start by uploading the PDF you want to explore. Upload the pdf/document to kaggle and then past the correct path to that pdf. This can be any document, such as a book, technical manual, research paper, or legal text.

### 2. Summarize the Document
After uploading, you can ask for a quick summary by typing "What is the summary of this file?" The AI will analyze the document and provide a concise overview of the content.

### 3. Ask Detailed Questions
Enter specific questions in the input box, such as "What is discussed in Chapter 2?" or "Explain the concept of neural networks mentioned in the book." The AI will retrieve relevant chunks and use the Gemini model to generate detailed answers.

### 4. Iterate and Explore
You can continue asking questions about different topics or specific sections as needed. If you’re looking for information on a particular keyword or phrase, ask targeted questions to get precise answers from the document.

### <span style="color:green">Install Required Dependencies:</span>
First, install the necessary libraries. Run the following cell to install them:

In [86]:
!pip install google-generativeai PyPDF2 sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


###  <span style="color:green">Import and use the API key</span>

In [87]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("GEMINI_API_KEY")

### <span style="color:green">Import Libraries and Set Up Google Vertex AI API Key:</span>
In this step, we’ll import all the libraries and set up your API key. Make sure you securely store the API key and never expose it publicly.

In [88]:
import os
import PyPDF2
from sentence_transformers import SentenceTransformer, util
import google.generativeai as genai
from typing import List, Dict

# Configure Vertex AI
genai.configure(api_key=secret_value)

##  <span style="color:green">Models and Its Purpose</span>
**Google Generative AI (Gemini-1.5-flash):** This model generates text-based responses to questions, enabling more conversational answers based on the PDF content.

**Sentence Transformer (all-MiniLM-L6-v2):** Creates semantic embeddings, which help locate relevant text snippets in the PDF for precise answers.

In [89]:
generation_config = {
    "temperature": 0.2,  # Adjust as needed
    "top_p": 0.95,       # Adjust as needed
    "top_k": 10,         # Adjust as needed
    "max_output_tokens": 1024,  # Adjust based on expected answer length
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Initialize sentence transformer model for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model

## <span style="color:green">BookChatbot Class:</span>
The BookChatbot class will handle loading the PDF, extracting text, generating embeddings, and answering questions.

In [90]:
import warnings
warnings.filterwarnings("ignore")

class BookChatbot:
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.chunks = []
        self.embeddings = []
        self.load_and_process_pdf()

    def load_and_process_pdf(self):
        # Read the PDF and split text into chunks
        reader = PyPDF2.PdfReader(self.pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        
        # Split into paragraphs for more coherent chunks
        paragraphs = text.split("\n\n")
        for paragraph in paragraphs:
            if paragraph.strip():  # Ignore empty paragraphs
                self.chunks.append(paragraph.strip())
        
        # Generate embeddings for each chunk
        self.embeddings = embedding_model.encode(self.chunks, convert_to_tensor=True)

    def answer_question(self, question: str, context_window: int = 5) -> str:
        # Retrieve relevant chunks based on semantic similarity
        relevant_chunks = self.find_relevant_chunks(question, context_window)
        
        # Prepare prompt with relevant context
        prompt = f"Answer the following question based on this context:\n\nContext:\n{relevant_chunks}\n\nQuestion:\n{question}\n\nAnswer:\n"
        response = model.generate_content([prompt])
        
        return response.text.strip()

    def find_relevant_chunks(self, question: str, context_window: int = 5) -> str:
        # Embed the question for semantic similarity search
        question_embedding = embedding_model.encode(question, convert_to_tensor=True)
        
        # Compute similarity scores between question and each chunk
        similarities = util.pytorch_cos_sim(question_embedding, self.embeddings)[0]
        
        # Ensure context_window is not greater than the number of chunks
        num_chunks = len(self.chunks)
        context_window = min(context_window, num_chunks)
        
        # Retrieve the most relevant chunks
        top_indices = similarities.topk(context_window).indices
        relevant_chunks = [self.chunks[idx] for idx in top_indices]
        
        return "\n".join(relevant_chunks)

## <span style="color:green">Input</span>

In [91]:
pdf_file = "/kaggle/input/ai-tot/artificial_intelligence_tutorial.pdf"

## <span style="color:green">Main</span>

In [ ]:
import os
import logging

# Suppress tqdm progress bars if any in the output.
os.environ["TQDM_DISABLE"] = "1"
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)

In [93]:
chatbot = BookChatbot(pdf_file)
summary = 1

while True:
    if summary == 1:
        Q = "What is the summary of this file/pdf, briefly explain."
        ans = chatbot.answer_question(Q)
        print("Summary of document:\n",ans,"\n")
        summary += 1
        
        print("Ask questions about the book (or type 'exit'):")
    
    user_question = input("Question:")
    if user_question.lower() == 'exit':
        print("Good Bye, see you soon!")
        break

    answer = chatbot.answer_question(user_question)

    display(Markdown("**Answer:** "+answer))
    print("\n")

Summary of document:
 This document is a comprehensive introduction to Artificial Intelligence (AI). It covers various aspects of AI, including its history, philosophy, goals, research areas, and applications. 

The document starts by defining AI and its key components like reasoning, learning, problem-solving, perception, and linguistic intelligence. It then delves into different research areas of AI, such as speech and voice recognition, expert systems, natural language processing, neural networks, and robotics. 

Each research area is explained in detail, including its components, working principles, advantages, disadvantages, and real-life applications. The document also discusses the concept of agents and environments in AI, and explores various search algorithms used for problem-solving. 

Finally, the document touches upon the ethical and societal implications of AI, highlighting potential threats to privacy, human dignity, and safety. It concludes with a glossary of frequently 

Question: total chapters?


**Answer:** There are **12** chapters in this document.

Question: tell me about chapter 10 in 4 sentences.


**Answer:** Chapter 10 of this tutorial focuses on Artificial Neural Networks (ANNs), which are inspired by the human brain's neural network. ANNs consist of interconnected nodes that process information and learn by adjusting their weights. The chapter discusses different types of ANNs, including feedforward and feedback networks, and explains how they work through learning algorithms like backpropagation. Finally, it highlights various applications of ANNs in diverse fields such as aerospace, automotive, military, and finance.

...

**Answer:** Semantic analysis in NLP is the process of extracting the meaning or dictionary meaning from the text. It checks the text for meaningfulness by mapping syntactic structures and objects in the task domain.

Question: what are the applications of expert system


**Answer:** Expert systems have a wide range of applications across various domains. Here are some key areas where they are used:

* **Design Domain:**  Camera lens design, automobile design, and other complex engineering designs.
* **Medical Domain:**  Diagnosis systems to deduce the cause of diseases from observed data, conducting medical operations on humans, and providing medical advice.
* **Monitoring Systems:**  Continuously comparing data with observed systems or prescribed behavior, such as leakage monitoring in long petroleum pipelines.
* **Process Control Systems:**  Controlling physical processes based on monitoring, like industrial automation and manufacturing.
* **Knowledge Domain:**  Finding out faults in vehicles, computers, and other complex systems.
* **Finance/Commerce:**  Detection of possible fraud, suspicious transactions, stock market trading, airline scheduling, cargo scheduling, and other financial and business applications.

Question: exit


Good Bye, see you soon!


## Conclusion
In this notebook, we explored the capabilities of our Book/document AI chat, designed to provide insightful answers based on the content of any uploaded PDF. With the integration of advanced neural network models and efficient text processing, we have created a tool that enhances learning and information retrieval. Thank you for exploring this project with me!

### If you found this notebook helpful, please **<span style="color:red">UPVOTE</span>**, 🌟 Your support motivates me to share more valuable insights. <span style="color:green">Thank You</span>.

### Will work further on it to accept the url of pdf as well, if anything can be improved please give a <span style="color:red">feedback</span>, would be grateful.

### Book (AI tutorial) I have used can be find on the attach url
- https://www.dcpehvpm.org/E-Content/BCA/BCA-III/artificial_intelligence_tutorial.pdf